In [ ]:
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential
from PIL import Image

client = ImageAnalysisClient(
    endpoint="",
    credential=AzureKeyCredential("")
)

base_width = 1000
img = Image.open('./frames/frame_0000.jpg')

wpercent = (base_width / float(img.size[0]))
hsize = int((float(img.size[1]) * float(wpercent)))
img = img.resize((base_width, hsize), Image.Resampling.LANCZOS)
img.save('./frames/frame_0000_.jpg')

with open("./frames/frame_0000_.jpg", "rb") as f:
    image_data = f.read()

result = client.analyze(
            image_data=image_data,
            visual_features=[VisualFeatures.OBJECTS]
        )

print("Image analysis results:")
print(" Objects:")

if result.objects is not None:
    for object in result.objects.list:
        print(f"   '{object.tags[0].name}', {object.bounding_box}, Confidence: {object.tags[0].confidence:.4f}")

print(f" Image height: {result.metadata.height}")
print(f" Image width: {result.metadata.width}")
print(f" Model version: {result.model_version}")

In [ ]:
import os
import time

import cv2
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential

client = ImageAnalysisClient(
    endpoint="",
    credential=AzureKeyCredential("")
)

# Path to the video file
video_path = './Data/test.mp4'
# Directory to save frames
output_dir = './frames'
os.makedirs(output_dir, exist_ok=True)

# Open the video file
cap = cv2.VideoCapture(video_path)
frame_count = 0

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break
    else:
        frame_path = os.path.join(output_dir, f'frame_{frame_count:04d}.jpg')
        cv2.imwrite(frame_path, frame)
        frame_count += 1
        time.sleep(0.1)
cap.release()
print(f'{frame_count} frames extracted.')

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials

computervision_client = ComputerVisionClient("", CognitiveServicesCredentials(""))

read_image = open("./frames/frame_0164.jpg", "rb")

# Call API with image and raw response (allows you to get the operation location)
read_response = computervision_client.read_in_stream(read_image, raw=True)
# Get the operation location (URL with ID as last appendage)
read_operation_location = read_response.headers["Operation-Location"]
# Take the ID off and use to get results
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for the retrieval of the results
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status.lower () not in ['notstarted', 'running']:
        break
    print ('Waiting for result...')
    time.sleep(10)

# Print results, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            print(line.bounding_box)

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials

import re
import time

read_image_url = "https://img.pistonheads.com/LargeSize/ford/focus-st/st-3/ford-focus-st-st-3-S4788196-1.jpg"
computervision_client = ComputerVisionClient("", CognitiveServicesCredentials(""))

# Regular expression for UK VRN
vrn_pattern = r'^[A-Z]{2}[0-9]{2}\s?[A-Z]{3}$|^[A-Z]{1,2}[0-9]{1,4}\s?[A-Z]{1,3}$|^[A-Z]{3}\s?[0-9]{1,4}$'

# Call API with URL and raw response (allows you to get the operation location)
read_response = computervision_client.read(read_image_url,  raw=True)

# Get the operation location (URL with an ID at the end) from the response
read_operation_location = read_response.headers["Operation-Location"]

# Grab the ID from the URL
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for it to retrieve the results 
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

# Print the detected text, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            if re.match(vrn_pattern, line.text):
                print(line.text)
                print(line.bounding_box)

In [ ]:
import cv2
import numpy as np

# Coordinates of the bounding box
points = np.array([[510, 1111], [791, 1113], [790, 1183], [510, 1180]], np.int32)
points = points.reshape((-1, 1, 2))

# Load your image
image = cv2.imread('path_to_your_image.jpg')

# Draw the bounding box
cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)

# Save or display the image
cv2.imwrite('output_image.jpg', image)
cv2.imshow('Image with Bounding Box', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
